In [1]:
# Load packages. 
library(loadeR)
library(transformeR)
library(loadeR.ECOMS)
library(visualizeR)
library(convertR)
library(drought4R)
library(downscaleR)

Loading required package: rJava
Loading required package: loadeR.java
Java version 11x amd64 by Ubuntu detected
NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready
loadeR version 1.4.15 (2019-07-30) is loaded
Get the latest stable version (1.6.0) using <devtools::install_github('SantanderMetGroup/loadeR')>
Please use 'citation("loadeR")' to cite this package.
transformeR version 1.6.1 (2019-11-01) is loaded
Please see 'citation("transformeR")' to cite this package.
loadeR.ECOMS version 1.4.6 (2018-10-20) is loaded
Please use 'citation("loadeR.ECOMS")' to cite this package.
Warning message:
“no DISPLAY variable so Tk is not available”visualizeR version 1.5.0 (2019-10-04) is loaded
Get the latest stable version (1.5.1) using <devtools::install_github('SantanderMetGroup/visualizeR')>
Please see 'citation("visualizeR")' to cite this package.

Attaching package: ‘visualizeR’

The following object is masked from ‘package:transformeR’:

    clim2sgdf

Loading required package: 

# WATExR climate data processing

This notebook aggregates the following R scripts (from [here](https://github.com/icra/WATExR/tree/master/R)):

 * `observations.R`
 * `reanalysis.R`
 * `seasonalForecast.R`

Modifications have been made to suit the Morsa case study in Norway.

The notebook rather messy and generates a lot of cell output, but it's useful to be able to run the whole workflow in one go. Note that downloading the data takes a long time (several hours), so it's probably best to run this e.g overnight.

## 1. User settings

In [2]:
# Output path where the data will be saved (change to your local path).
dir.data <- '/home/jovyan/projects/watexr/data/' 
dir.Rdata <- '/home/jovyan/projects/watexr/Rdata/'
  
# Define the geographical domain for the Morsa catchment
latLim <- c(59.31, 59.90) 
lonLim <- c(10.63, 11.25) 

# Define the coordinates and name of the lake
lake <- list(x = 10.895, y = 59.542) # Roughly the middle of Morsa catchment
lakename <- "Morsa"

# Define the period and the season
years <- 1981:2010
season <- 1:12 # Full year

# Login in the TAP-UDG the climate4R libraries 
# More details about UDG in https://doi.org/10.1016/j.cliser.2017.07.001
loginUDG("WATExR", "1234567890")

# Define metadata to generate the file name
institution <- "NIVA"
lake_id <- lakename

[2020-01-15 12:39:00] Setting credentials...
[2020-01-15 12:39:00] Success!
Go to <http://www.meteo.unican.es/udg-tap/home> for details on your authorized groups and datasets


## 2. Download EWEMBI

Download historic, gridded, observational data (EWEMBI) for the Morsa catchment. The code is modified from the original [here](https://github.com/icra/WATExR/blob/master/R/observations.R).

### 2.1. Select dataset

In [ ]:
# Define the dataset 
dataset <- "PIK_Obs-EWEMBI"

# Check available variables in the dataset (EWEMBI)  
di <- dataInventory(dataset)
names(di)

### 2.2. Data loading and transformation

In [ ]:
# Define the variables to be loaded. Remove those not needed. 
variables <- c("uas", "vas", "ps", "tas", "pr", "rsds", "rlds", "hurs")

# Load observations (EWEMBI) with function loadGridData from package loadeR.
# Data is loaded in a loop (function lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable
data.prelim <- lapply(variables, function(x) loadGridData(dataset, var = x, years = years, 
                                                   lonLim = lonLim, latLim = latLim, 
                                                   season = season))
names(data.prelim) <- variables

# Bilinear interpolation of the data to the location of the lake. See ?interpGrid for other methods.
data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
                                                   method = "bilinear", 
                                                   bilin.method = "akima"))

#Convert pressure units to millibars with function udConvertGrid from package convertR.
data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

# Collect some common metadata (e.g. from variable uas)
dates <- data.interp[[1]]$Dates
xycoords <- getCoordinates(data.interp[[1]])

## Compute cloud cover with function rad2cc from package convertR
#clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
#clt$Variable$varName <- "cc"
#
## Put all variables together
#data <- c(data.interp, "cc" = list(clt))
data <- data.interp

############################################################################################
############### RUN THE FOLLOWING CODE CHUNK IF YOU NEED POTENTIAL EVAPOTRANSPIRATION ######
# Load needed variables 
tasmin <- loadGridData(dataset, var = "tasmin", years = years, 
                       lonLim = lonLim, latLim = latLim, 
                       season = season,  time = "DD", aggr.d = "min")
tasmax <- loadGridData(dataset, var = "tasmax", years = years, 
                       lonLim = lonLim, latLim = latLim, 
                       season = season,  time = "DD", aggr.d = "max")

# Compute potential evapotranspiration with function petGrid from package drought4R
# For daily data the implemented method is hargreaves-samani (See ?petGrid for details):
petH <- petGrid(tasmin = tasmin, 
                tasmax = tasmax,
                method = "hargreaves-samani")

# bilinear interpolation 
petH.interp <- interpGrid(petH, new.coordinates = lake, method = "bilinear", bilin.method = "akima")
petH.interp$Variable$varName <- "petH"

# Put all variables together
data <- c(data, "petH" = list(petH.interp))
###################### END OF THE CHUNK ####################################################
############################################################################################

### 2.3. Save results 

In [ ]:
# save Rdata for posterior bias correction of seasonal forecasts
save(data, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), ".rda"))

# extract the data arrays of all variables from the list
data <- lapply(data, function(x) x[["Data"]])
# Remove unwanted variables from output
data["rsds"] <- NULL 
data["rlds"] <- NULL
# Build data frame
yymmdd <- as.Date(dates$start)
hhmmss <- format(as.POSIXlt(dates$start), format = "%H:%M:%S") 
df <- data.frame(c(list("dates1" = yymmdd, "dates2" = hhmmss)), data)

########### EXPORT DATA ACCORDING TO THE WATExR ARCHIVE DESIGN -----------------------------
## SEE the proposal for the WATExR Archive Design in:                                            
## https://docs.google.com/document/d/1yzNtw9W_z_ziPQ6GrnSgD9ov5O1swnohndDTAWOgpwc/edit

# Define metadata to generate the file name
ClimateModelName <- "EWEMBI"
ExperimentName <- "observations"
member <- "member01"
freq <- "day"

# Create directory and save file
startTime <- format(as.POSIXlt(yymmdd[1]), format = "%Y%m%d")
endTime <- format(tail(as.POSIXlt(yymmdd), n = 1), format = "%Y%m%d")
dirName <- paste0(dir.data, lake_id, "/CLIMATE/", lake_id, "_", institution, "_", ClimateModelName, "_", ExperimentName, "_", member, "_", freq, "_", startTime, "-", endTime, "/", sep = "", collapse = NULL)
dir.create(dirName, showWarnings = TRUE, recursive = TRUE, mode = "0777")
write.table(df, paste0(dirName,"meteo_file.dat", sep = "", collapse = NULL), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

## 3. Download and bias correct ERA-Interim

Download historic, gridded, reanalysis data (ERA-Interim) for the Morsa catchment. The code is modified from the original [here](https://github.com/icra/WATExR/blob/master/R/reanalysis.R).

### 3.1. Select dataset

In [ ]:
## dataset <- "ECMWF_ERA-Interim-ESD"
dataset <- "http://meteo.unican.es/tds5/dodsC/interim/interim075_WATExR.ncml"

# Check available variables in the dataset (ERA-Interim)  
di <- dataInventory(dataset)
names(di)

### 3.2. Data loading and transformation

In [ ]:
# Path to the observational data (change to your local path).
dir.Rdata.obs <- "/home/jovyan/projects/watexr/Rdata/PIK_Obs-EWEMBI_1_2_3_4_5_6_7_8_9_10_11_12_uas_vas_ps_tas_pr_rsds_rlds_hurs_petH.rda"
obs.data <- get(load(dir.Rdata.obs))

# Define the variables to be loaded (the same as in the observational data, 
# except clould cover (cc) and evapotranspiration (petH))
varnames.obs <- sapply(obs.data, function(x) getVarNames(x)) # to check the variables in the observational data.
varnames.obs

In [ ]:
# Define the variables to be loaded. Remove those not needed.
variables <- c("uas", "vas", "ps", "tas", "pr", "rsds", "rlds")
                       
# Define daily aggregation function for each variable selected. 
aggr.fun <- c("mean", "mean", "mean", "mean", "sum", "mean", "mean")

# Load reanalysis (ERA-Interim) with function loadGridData from package loadeR.
# Data is loaded in a loop (function lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable
data.prelim <- lapply(1:length(variables), function(x) loadGridData(dataset, var = variables[x], years = years, 
                                                                    lonLim = lonLim, latLim = latLim, season = season, 
                                                                    time = "DD", aggr.d = aggr.fun[x]))

# Deal with the special case of accumulated variables (get temporal intersection)
data.prelim <- intersectGrid(data.prelim, type = "temporal", which.return = 1:length(variables))
names(data.prelim) <- c("uas", "vas", "ps", "tas", "pr", "rsds", "rlds")

# Compute relative humidity from the mean temperature and the dew point with function tdps2hurs from package convertR
tdps <- loadGridData(dataset, var = "tdps", years = years, 
                     lonLim = lonLim, latLim = latLim, 
                     season = season,  time = "DD", aggr.d = "mean")
tdps <- intersectGrid(tdps, data.prelim$tas, which.return = 1)

hurs <- data.prelim$tas # Predefine the object
hurs$Data <- tdps2hurs(data.prelim$tas$Data, tdps$Data) # Assign the data matrix
# Define correctly the metadata of the object:
hurs$Variable$varName <- "hurs"
attr(hurs$Variable,"units") <- "%"
attr(hurs$Variable,"description") <- "2 metre relative humidity"
attr(hurs$Variable,"longname") <- "hurs"
# Include variables in data.prelim
data.prelim <- c(data.prelim, "hurs" = list(hurs))

# Compute wss
wss <- data.prelim$uas
wss$Data <- data.prelim$uas$Data^2 + data.prelim$vas$Data^2
# Define correctly the metadata of the object:
wss$Variable$varName <- "wss"
attr(wss$Variable,"units") <- "m s**-1"
attr(wss$Variable,"description") <- "Near-Surface Wind Speed"
attr(wss$Variable,"longname") <- "wss"
# Include variables in data.prelim
data.prelim <- c(data.prelim, "wss" = list(wss))

# Bilinear interpolation of the data to the location of the lake. See ?interpGrid for other methods.
data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
                                                          method = "bilinear", 
                                                          bilin.method = "akima"))

# Convert pressure and temperature units to millibars and celsius with function udConvertGrid from package convertR.
data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars") #No need SWAT
data.interp$tas <- udConvertGrid(data.interp$tas, new.units = "celsius")

# Convert radiation units from J/m2/12hours to W/m2
data.interp$rsds$Data <- data.interp$rsds$Data/43200 
attr(data.interp$rsds$Variable,"units") <- "W.m-2"
data.interp$rlds$Data <- data.interp$rlds$Data/43200 
attr(data.interp$rlds$Variable,"units") <- "W.m-2"

#Convert relative humidity units to fractions with function udConvertGrid from package convertR.
data.interp$hurs <- udConvertGrid(data.interp$hurs, new.units = "")

#Convert shortwave radiation units to MJ/(m2*day) with function udConvertGrid from package convertR.
data.interp$rsds <- udConvertGrid(data.interp$rsds, new.units = "MJ m-2 day-1")
data.interp$rlds <- udConvertGrid(data.interp$rlds, new.units = "MJ m-2 day-1")

## Compute cloud cover with function rad2cc
#clt <- redim(rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds), drop = TRUE)
#clt$Variable$varName <- "cc"
#
## Put all variables together
#data <- c(data.interp, "cc" = list(clt))
data <- data.interp

############################################################################################
############### RUN THE FOLLOWING CODE CHUNK IF YOU NEED POTENTIAL EVAPOTRANSPIRATION ######
# Load needed variables 
tasmin <- loadGridData(dataset, var = "tas", years = years, 
                       lonLim = lonLim, latLim = latLim, 
                       season = season,  time = "DD", aggr.d = "min")
tasmax <- loadGridData(dataset, var = "tas", years = years, 
                       lonLim = lonLim, latLim = latLim, 
                       season = season,  time = "DD", aggr.d = "max")
# Compute potential evapotranspiration with function petGrid from package drought4R
# For daily data the implemented method is hargreaves-samani (See ?petGrid for details)
# petGrid function requires temperature in celsius. Convert temperature units to celsius.
tasmax <- udConvertGrid(tasmax, new.units = "celsius")
tasmin <- udConvertGrid(tasmin, new.units = "celsius")
petH <- petGrid(tasmin = tasmin, 
                tasmax = tasmax,
                method = "hargreaves-samani")

# bilinear interpolation 
petH.interp <- interpGrid(petH, new.coordinates = lake, method = "bilinear", bilin.method = "akima")
petH.interp$Variable$varName <- "petH"

# Put all variables together
data <- c(data, "petH" = list(petH.interp))
###################### END OF THE CHUNK ####################################################
############################################################################################

### 3.3. Bias correction

In [ ]:
# Check variable consistency
if (!all(names(obs.data) %in% names(data))) stop("variables in obs.data and data (seasonal forecast) do not match.")

#order variables
data <- data[match(names(obs.data), names(data))]
varnames <- names(data)

# Subset observational data to the same dates as forecast data
obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)})
data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 2)})
names(obs.data) <- varnames
names(data) <- varnames

# Collect some common metadata (e.g. from variable uas)
dates <- data[[1]]$Dates
xycoords <- getCoordinates(data[[1]])

# Bias correction with leave-one-year-out ("loo") cross-validation
# type ?biasCorrection in R for more info about the parameter settings for bias correction.
data.bc.cross <- lapply(1:length(data), function(x)  {
  precip <- FALSE
  if (names(data)[x] == "pr") precip <- TRUE
  biasCorrection(y = obs.data[[x]], x = data[[x]], 
                 method = "eqm", cross.val = "loo",
                 precipitation = precip,
                 wet.threshold = 1,
                 window = c(90, 31),
                 join.members = TRUE)
}) 
names(data.bc.cross) <- varnames

# Bias correction without cross-validation
data.bc <- lapply(1:length(data), function(v)  {
  pre <- FALSE
  print(names(data)[v])
  if (names(data)[v] == "pr") pre <- TRUE
  biasCorrection(y = obs.data[[v]], x = data[[v]], 
                 method = "eqm",
                 precipitation = pre,
                 wet.threshold = 1,
                 window = c(90, 31),
                 join.members = TRUE)
}) 
names(data.bc) <- varnames

### 3.4. Save results

In [ ]:
# save Rdata (*.rda file)
save(data, file = paste0(dir.Rdata, "interim075_WATExR_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_raw.rda"))
save(data.bc.cross, file = paste0(dir.Rdata, "interim075_WATExR_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BCcross.rda"))
save(data.bc, file = paste0(dir.Rdata, "interim075_WATExR_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BC.rda"))

datatoexport <- data.bc

# extract the data arrays of all variables from the list
data <- lapply(datatoexport, function(x) x[["Data"]])

# Remove unwanted variables from output
data["rsds"] <- NULL 
data["rlds"] <- NULL
# Build data frame
yymmdd <- as.Date(dates$start)
hhmmss <- format(as.POSIXlt(dates$start), format = "%H:%M:%S") 
df <- data.frame(c(list("dates1" = yymmdd, "dates2" = hhmmss)), data)

########### EXPORT DATA ACCORDING TO THE WATExR ARCHIVE DESIGN -----------------------------
## SEE the proposal for the WATExR Archive Design in:                                            
## https://docs.google.com/document/d/1yzNtw9W_z_ziPQ6GrnSgD9ov5O1swnohndDTAWOgpwc/edit

# Define metadata to generate the file name
ClimateModelName <- "ERA-Interim"
ExperimentName <- "reanalysis"
member <- "member01"
freq <- "day"

# Create directory and save file
startTime <- format(as.POSIXlt(yymmdd[1]), format = "%Y%m%d")
endTime <- format(tail(as.POSIXlt(yymmdd), n = 1), format = "%Y%m%d")
dirName <- paste0(dir.data, lake_id, "/CLIMATE/", lake_id, "_", institution, "_", ClimateModelName, "_", ExperimentName, "_", member, "_", freq, "_", startTime, "-", endTime, "/", sep = "", collapse = NULL)
dir.create(dirName, showWarnings = TRUE, recursive = TRUE, mode = "0777")
write.table(df, paste0(dirName,"meteo_file.dat", sep = "", collapse = NULL), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

## 4. Download and bias correct seasonal forecast (S4) data

Download gridded, seasonal forecast data (S4 15-member ensemble) for the Morsa catchment. The code is modified from the original [here](https://github.com/icra/WATExR/blob/master/R/seasonalForecast.R).

**Note:** The code in this section is currently repeated **four times**, which is messy. This should be tidied/refactored.

### 4.1. Winter

#### 4.1.1. Select dataset

In [3]:
options(java.parameters = "-Xmx8000m")

# Define the members
mem <- 1:15
# Define the lead month
lead.month <- 0
# Define period and season
season <- c(11,12,1) # Winter

# Define the dataset 
dataset <- "System4_seasonal_15" # or "CFSv2_seasonal"

# Check available variables in the dataset (System4)  
di <- dataInventory("http://www.meteo.unican.es/tds5/dodsC/system4/System4_Seasonal_15Members.ncml") # or "http://meteo.unican.es/tds5/dodsC/cfsrr/CFSv2_Seasonal.ncml"
names(di)

[2020-01-15 12:39:11] Doing inventory ...
[2020-01-15 12:39:35] Retrieving info for 'z1000mb' (45 vars remaining)
[2020-01-15 12:39:36] Retrieving info for 'z700mb' (44 vars remaining)
[2020-01-15 12:39:36] Retrieving info for 'z850mb' (43 vars remaining)
[2020-01-15 12:39:38] Retrieving info for 'z500mb' (42 vars remaining)
[2020-01-15 12:39:38] Retrieving info for 'z300mb' (41 vars remaining)
[2020-01-15 12:39:39] Retrieving info for 'z200mb' (40 vars remaining)
[2020-01-15 12:39:39] Retrieving info for 't1000mb' (39 vars remaining)
[2020-01-15 12:39:40] Retrieving info for 't700mb' (38 vars remaining)
[2020-01-15 12:39:41] Retrieving info for 't850mb' (37 vars remaining)
[2020-01-15 12:39:41] Retrieving info for 't500mb' (36 vars remaining)
[2020-01-15 12:39:42] Retrieving info for 't300mb' (35 vars remaining)
[2020-01-15 12:39:42] Retrieving info for 't200mb' (34 vars remaining)
[2020-01-15 12:39:43] Retrieving info for 'u1000mb' (33 vars remaining)
[2020-01-15 12:39:44] Retrieving

[1] "z1000mb"  "z700mb"   "z850mb"   "z500mb"   "z300mb"   "z200mb"  
 [7] "t1000mb"  "t700mb"   "t850mb"   "t500mb"   "t300mb"   "t200mb"  
[13] "u1000mb"  "u700mb"   "u850mb"   "u500mb"   "u300mb"   "u200mb"  
[19] "v1000mb"  "v700mb"   "v850mb"   "v500mb"   "v300mb"   "v200mb"  
[25] "q1000mb"  "q700mb"   "q850mb"   "q500mb"   "q300mb"   "q200mb"  
[31] "zsfc"     "ssrd"     "strd"     "tcc"      "u10m"     "v10m"    
[37] "dpt2m"    "t2m"      "mean2t24" "sst"      "mx2t24"   "mn2t24"  
[43] "sd"       "sf"       "tp"       "mslp"

#### 4.1.2. Data loading and transformation

In [ ]:
# Path to the observational data (change to your local path).
dir.Rdata.obs <- "/home/jovyan/projects/watexr/Rdata/PIK_Obs-EWEMBI_1_2_3_4_5_6_7_8_9_10_11_12_uas_vas_ps_tas_pr_rsds_rlds_hurs_petH.rda"
obs.data <- get(load(dir.Rdata.obs))

# Define the variables to be loaded (the same as in the observational data, 
# except clould cover (cc) and evapotranspiration (petH))
sapply(obs.data, function(x) getVarNames(x)) # to check the variables in the observational data.
variables <- c("uas", "vas", "ps", "tas", "pr", "rsds", "rlds", "hurs")

# Define daily aggregation function for each variable selected
aggr.fun <- c("mean", "mean", "mean", "mean", "sum", "mean", "mean", "mean")

# Load seasonal forecast data (System4 or CFS) with function loadECOMS
# Data is loaded in a loop (función lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable
data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
                                                          members = mem, leadMonth = lead.month,
                                                          lonLim = lonLim, latLim = latLim, season = season, 
                                                          time = "DD", aggr.d = aggr.fun[x]))
names(data.prelim) <- variables

# Bilinear interpolation of the data to the location of the lake
data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
                                                          method = "bilinear", 
                                                          bilin.method = "akima"))

# Convert pressure units to millibars with function udConvertGrid from package convertR.
data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

## Compute cloud cover with function rad2cc
#clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
#clt$Variable$varName <- "cc"
#
## Put all variables together
#data <- c(data.interp, "cc" = list(clt))
data <- data.interp

############################################################################################
############### RUN THE FOLLOWING CODE CHUNK IF YOU NEED POTENTIAL EVAPOTRANSPIRATION ######
# Load needed variables 
tasmin <- loadECOMS(dataset, var = "tasmin", years = years, 
                    lonLim = lonLim, latLim = latLim, 
                    leadMonth = lead.month, members = mem,
                    season = season, time = "DD", aggr.d = "min")
tasmax <- loadECOMS(dataset, var = "tasmax", years = years, 
                    lonLim = lonLim, latLim = latLim, 
                    leadMonth = lead.month, members = mem,
                    season = season, time = "DD", aggr.d = "max")

# Compute potential evapotranspiration with function petGrid from package drought4R
# For daily data the implemented method is hargreaves-samani (See ?petGrid for details):
petH <- petGrid(tasmin = tasmin, 
                tasmax = tasmax,
                method = "hargreaves-samani")

# bilinear interpolation 
petH.interp <- interpGrid(petH, new.coordinates = lake, method = "bilinear", bilin.method = "akima")
petH.interp$Variable$varName <- "petH"

# Put all variables together
data <- c(data, "petH" = list(petH.interp))
###################### END OF THE CHUNK ####################################################
############################################################################################

#### 4.1.3. Bias correction

In [ ]:
# Subset all datasets to the same Dates as the hindcast precipitation. Note that we compute daily accumulated 
# precipitation, for this reason this variable has no value for the first day of every season.  
if (sum(names(data)=="pr")>0){
  data <- lapply(1:length(data), function(x)  {intersectGrid(data[[x]], data[[which(names(data)=="pr")]], type = "temporal", which.return = 1)}) 
  names(data) <- sapply(data, function(x) getVarNames(x))
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))
} else{
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))  
}

# Check variable consistency
if (!identical(names(obs.data), names(data))) stop("variables in obs.data and data (seasonal forecast) do not match.")

# Bias correction with leave-one-year-out ("loo") cross-validation
# type ?biasCorrection in R for more info about the parameter settings for bias correction.
data.bc.cross <- lapply(1:length(data), function(x)  {
    precip <- FALSE
    if (names(data)[x] == "pr") precip <- TRUE
    biasCorrection(y = obs.data[[x]], x = data[[x]], 
                method = "eqm", cross.val = "loo",
                precipitation = precip,
                wet.threshold = 1,
                join.members = TRUE)
  }) 
names(data.bc.cross) <- names(data)
                            
# Bias correction without cross-validation
data.bc <- lapply(1:length(data), function(v)  {
    pre <- FALSE
    if (names(data)[v] == "pr") pre <- TRUE
    biasCorrection(y = obs.data[[v]], x = data[[v]], 
                 method = "eqm",
                 precipitation = pre,
                 wet.threshold = 1,
                 join.members = TRUE)
}) 
names(data.bc) <- names(data) 

#### 4.1.4. Save results

In [ ]:
# save Rdata (*.rda file)
save(data, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_raw.rda"))
save(data.bc.cross, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BCcross.rda"))
save(data.bc, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BC.rda"))

########## BUILD FINAL DATA AND EXPORT ACCORDING TO THE WATExR ARCHIVE DESIGN -----------------------
## SEE the proposal for the WATExR Archive Design in:                                            
## https://docs.google.com/document/d/1yzNtw9W_z_ziPQ6GrnSgD9ov5O1swnohndDTAWOgpwc/edit

# Select the object to export (can be 'data.bc', 'data.bc.cross' or 'data')
datatoexport <- data.bc

# Collect some common metadata (e.g. from variable uas)
dates <- datatoexport[[1]]$Dates
xycoords <- getCoordinates(datatoexport[[1]])

# Give format to dates
yymmdd <- as.Date(dates$start)
hhmmss <- format(as.POSIXlt(dates$start), format = "%H:%M:%S") 

# Define metadata to generate the file name
ClimateModelName <- dataset
ExperimentName <- "seasonal"
freq <- "day"

# Save a single file for each member
for (i in mem) {
  # Build data.frame for a single member
  single.member <- lapply(datatoexport, function(x) subsetGrid(x, members = i))
  single.member <- lapply(single.member, function(x) x$Data)
  # Remove unwanted variables
  single.member["rsds"] <- NULL
  single.member["rlds"] <- NULL
  # data.frame creation
  df <- data.frame(c(list("dates1" = yymmdd, "dates2" = hhmmss)), single.member)
  if (i < 10) {
    member <- paste0("member0", i, sep = "", collapse = NULL)
  } else {
    member <- paste0("member", i, sep = "", collapse = NULL)
  }    
  startTime <- format(as.POSIXlt(yymmdd[1]), format = "%Y%m%d")
  endTime <- format(tail(as.POSIXlt(yymmdd), n = 1), format = "%Y%m%d")
  dirName <- paste0(dir.data, lake_id, "/CLIMATE/", lake_id, "_", institution, "_", ClimateModelName, "_", ExperimentName, "_", member, "_", freq,"_", startTime, "-", endTime, "/", sep = "", collapse = NULL)
  dir.create(dirName, showWarnings = TRUE, recursive = TRUE, mode = "0777")
  write.table(df, paste0(dirName, "meteo_file.dat", sep = "", collapse = NULL), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
}

### 4.2. Spring

#### 4.2.1. Select dataset

In [ ]:
options(java.parameters = "-Xmx8000m")

# Define the members
mem <- 1:15
# Define the lead month
lead.month <- 0
# Define period and season
season <- c(2,3,4)   # Spring

# Define the dataset 
dataset <- "System4_seasonal_15" # or "CFSv2_seasonal"

# Check available variables in the dataset (System4)  
di <- dataInventory("http://www.meteo.unican.es/tds5/dodsC/system4/System4_Seasonal_15Members.ncml") # or "http://meteo.unican.es/tds5/dodsC/cfsrr/CFSv2_Seasonal.ncml"
names(di)

#### 4.2.2. Data loading and transformation

In [ ]:
# Path to the observational data (change to your local path).
dir.Rdata.obs <- "/home/jovyan/projects/watexr/Rdata/PIK_Obs-EWEMBI_1_2_3_4_5_6_7_8_9_10_11_12_uas_vas_ps_tas_pr_rsds_rlds_hurs_petH.rda"
obs.data <- get(load(dir.Rdata.obs))

# Define the variables to be loaded (the same as in the observational data, 
# except clould cover (cc) and evapotranspiration (petH))
sapply(obs.data, function(x) getVarNames(x)) # to check the variables in the observational data.
variables <- c("uas", "vas", "ps", "tas", "pr", "rsds", "rlds", "hurs")

# Define daily aggregation function for each variable selected
aggr.fun <- c("mean", "mean", "mean", "mean", "sum", "mean", "mean", "mean")

# Load seasonal forecast data (System4 or CFS) with function loadECOMS
# Data is loaded in a loop (función lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable
data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
                                                          members = mem, leadMonth = lead.month,
                                                          lonLim = lonLim, latLim = latLim, season = season, 
                                                          time = "DD", aggr.d = aggr.fun[x]))
names(data.prelim) <- variables

# Bilinear interpolation of the data to the location of the lake
data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
                                                          method = "bilinear", 
                                                          bilin.method = "akima"))

# Convert pressure units to millibars with function udConvertGrid from package convertR.
data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

## Compute cloud cover with function rad2cc
#clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
#clt$Variable$varName <- "cc"
#
## Put all variables together
#data <- c(data.interp, "cc" = list(clt))
data <- data.interp

############################################################################################
############### RUN THE FOLLOWING CODE CHUNK IF YOU NEED POTENTIAL EVAPOTRANSPIRATION ######
# Load needed variables 
tasmin <- loadECOMS(dataset, var = "tasmin", years = years, 
                    lonLim = lonLim, latLim = latLim, 
                    leadMonth = lead.month, members = mem,
                    season = season, time = "DD", aggr.d = "min")
tasmax <- loadECOMS(dataset, var = "tasmax", years = years, 
                    lonLim = lonLim, latLim = latLim, 
                    leadMonth = lead.month, members = mem,
                    season = season, time = "DD", aggr.d = "max")

# Compute potential evapotranspiration with function petGrid from package drought4R
# For daily data the implemented method is hargreaves-samani (See ?petGrid for details):
petH <- petGrid(tasmin = tasmin, 
                tasmax = tasmax,
                method = "hargreaves-samani")

# bilinear interpolation 
petH.interp <- interpGrid(petH, new.coordinates = lake, method = "bilinear", bilin.method = "akima")
petH.interp$Variable$varName <- "petH"

# Put all variables together
data <- c(data, "petH" = list(petH.interp))
###################### END OF THE CHUNK ####################################################
############################################################################################

#### 4.2.3. Bias correction

In [ ]:
# Subset all datasets to the same Dates as the hindcast precipitation. Note that we compute daily accumulated 
# precipitation, for this reason this variable has no value for the first day of every season.  
if (sum(names(data)=="pr")>0){
  data <- lapply(1:length(data), function(x)  {intersectGrid(data[[x]], data[[which(names(data)=="pr")]], type = "temporal", which.return = 1)}) 
  names(data) <- sapply(data, function(x) getVarNames(x))
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))
} else{
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))  
}

# Check variable consistency
if (!identical(names(obs.data), names(data))) stop("variables in obs.data and data (seasonal forecast) do not match.")

# Bias correction with leave-one-year-out ("loo") cross-validation
# type ?biasCorrection in R for more info about the parameter settings for bias correction.
data.bc.cross <- lapply(1:length(data), function(x)  {
    precip <- FALSE
    if (names(data)[x] == "pr") precip <- TRUE
    biasCorrection(y = obs.data[[x]], x = data[[x]], 
                method = "eqm", cross.val = "loo",
                precipitation = precip,
                wet.threshold = 1,
                join.members = TRUE)
  }) 
names(data.bc.cross) <- names(data)
                            
# Bias correction without cross-validation
data.bc <- lapply(1:length(data), function(v)  {
    pre <- FALSE
    if (names(data)[v] == "pr") pre <- TRUE
    biasCorrection(y = obs.data[[v]], x = data[[v]], 
                 method = "eqm",
                 precipitation = pre,
                 wet.threshold = 1,
                 join.members = TRUE)
}) 
names(data.bc) <- names(data) 

#### 4.2.4. Save results

In [ ]:
# save Rdata (*.rda file)
save(data, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_raw.rda"))
save(data.bc.cross, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BCcross.rda"))
save(data.bc, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BC.rda"))

########## BUILD FINAL DATA AND EXPORT ACCORDING TO THE WATExR ARCHIVE DESIGN -----------------------
## SEE the proposal for the WATExR Archive Design in:                                            
## https://docs.google.com/document/d/1yzNtw9W_z_ziPQ6GrnSgD9ov5O1swnohndDTAWOgpwc/edit

# Select the object to export (can be 'data.bc', 'data.bc.cross' or 'data')
datatoexport <- data.bc

# Collect some common metadata (e.g. from variable uas)
dates <- datatoexport[[1]]$Dates
xycoords <- getCoordinates(datatoexport[[1]])

# Give format to dates
yymmdd <- as.Date(dates$start)
hhmmss <- format(as.POSIXlt(dates$start), format = "%H:%M:%S") 

# Define metadata to generate the file name
ClimateModelName <- dataset
ExperimentName <- "seasonal"
freq <- "day"

# Save a single file for each member
for (i in mem) {
  # Build data.frame for a single member
  single.member <- lapply(datatoexport, function(x) subsetGrid(x, members = i))
  single.member <- lapply(single.member, function(x) x$Data)
  # Remove unwanted variables
  single.member["rsds"] <- NULL
  single.member["rlds"] <- NULL
  # data.frame creation
  df <- data.frame(c(list("dates1" = yymmdd, "dates2" = hhmmss)), single.member)
  if (i < 10) {
    member <- paste0("member0", i, sep = "", collapse = NULL)
  } else {
    member <- paste0("member", i, sep = "", collapse = NULL)
  }    
  startTime <- format(as.POSIXlt(yymmdd[1]), format = "%Y%m%d")
  endTime <- format(tail(as.POSIXlt(yymmdd), n = 1), format = "%Y%m%d")
  dirName <- paste0(dir.data, lake_id, "/CLIMATE/", lake_id, "_", institution, "_", ClimateModelName, "_", ExperimentName, "_", member, "_", freq,"_", startTime, "-", endTime, "/", sep = "", collapse = NULL)
  dir.create(dirName, showWarnings = TRUE, recursive = TRUE, mode = "0777")
  write.table(df, paste0(dirName, "meteo_file.dat", sep = "", collapse = NULL), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
}

### 4.3. Early summer

#### 4.3.1. Select dataset

In [ ]:
options(java.parameters = "-Xmx8000m")

# Define the members
mem <- 1:15
# Define the lead month
lead.month <- 0
# Define period and season
season <- c(5,6,7)   # Early summer

# Define the dataset 
dataset <- "System4_seasonal_15" # or "CFSv2_seasonal"

# Check available variables in the dataset (System4)  
di <- dataInventory("http://www.meteo.unican.es/tds5/dodsC/system4/System4_Seasonal_15Members.ncml") # or "http://meteo.unican.es/tds5/dodsC/cfsrr/CFSv2_Seasonal.ncml"
names(di)

#### 4.3.2. Data loading and transformation

In [ ]:
# Path to the observational data (change to your local path).
dir.Rdata.obs <- "/home/jovyan/projects/watexr/Rdata/PIK_Obs-EWEMBI_1_2_3_4_5_6_7_8_9_10_11_12_uas_vas_ps_tas_pr_rsds_rlds_hurs_petH.rda"
obs.data <- get(load(dir.Rdata.obs))

# Define the variables to be loaded (the same as in the observational data, 
# except clould cover (cc) and evapotranspiration (petH))
sapply(obs.data, function(x) getVarNames(x)) # to check the variables in the observational data.
variables <- c("uas", "vas", "ps", "tas", "pr", "rsds", "rlds", "hurs")

# Define daily aggregation function for each variable selected
aggr.fun <- c("mean", "mean", "mean", "mean", "sum", "mean", "mean", "mean")

# Load seasonal forecast data (System4 or CFS) with function loadECOMS
# Data is loaded in a loop (función lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable
data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
                                                          members = mem, leadMonth = lead.month,
                                                          lonLim = lonLim, latLim = latLim, season = season, 
                                                          time = "DD", aggr.d = aggr.fun[x]))
names(data.prelim) <- variables

# Bilinear interpolation of the data to the location of the lake
data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
                                                          method = "bilinear", 
                                                          bilin.method = "akima"))

# Convert pressure units to millibars with function udConvertGrid from package convertR.
data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

## Compute cloud cover with function rad2cc
#clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
#clt$Variable$varName <- "cc"
#
## Put all variables together
#data <- c(data.interp, "cc" = list(clt))
data <- data.interp

############################################################################################
############### RUN THE FOLLOWING CODE CHUNK IF YOU NEED POTENTIAL EVAPOTRANSPIRATION ######
# Load needed variables 
tasmin <- loadECOMS(dataset, var = "tasmin", years = years, 
                    lonLim = lonLim, latLim = latLim, 
                    leadMonth = lead.month, members = mem,
                    season = season, time = "DD", aggr.d = "min")
tasmax <- loadECOMS(dataset, var = "tasmax", years = years, 
                    lonLim = lonLim, latLim = latLim, 
                    leadMonth = lead.month, members = mem,
                    season = season, time = "DD", aggr.d = "max")

# Compute potential evapotranspiration with function petGrid from package drought4R
# For daily data the implemented method is hargreaves-samani (See ?petGrid for details):
petH <- petGrid(tasmin = tasmin, 
                tasmax = tasmax,
                method = "hargreaves-samani")

# bilinear interpolation 
petH.interp <- interpGrid(petH, new.coordinates = lake, method = "bilinear", bilin.method = "akima")
petH.interp$Variable$varName <- "petH"

# Put all variables together
data <- c(data, "petH" = list(petH.interp))
###################### END OF THE CHUNK ####################################################
############################################################################################

#### 4.3.3. Bias correction

In [ ]:
# Subset all datasets to the same Dates as the hindcast precipitation. Note that we compute daily accumulated 
# precipitation, for this reason this variable has no value for the first day of every season.  
if (sum(names(data)=="pr")>0){
  data <- lapply(1:length(data), function(x)  {intersectGrid(data[[x]], data[[which(names(data)=="pr")]], type = "temporal", which.return = 1)}) 
  names(data) <- sapply(data, function(x) getVarNames(x))
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))
} else{
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))  
}

# Check variable consistency
if (!identical(names(obs.data), names(data))) stop("variables in obs.data and data (seasonal forecast) do not match.")

# Bias correction with leave-one-year-out ("loo") cross-validation
# type ?biasCorrection in R for more info about the parameter settings for bias correction.
data.bc.cross <- lapply(1:length(data), function(x)  {
    precip <- FALSE
    if (names(data)[x] == "pr") precip <- TRUE
    biasCorrection(y = obs.data[[x]], x = data[[x]], 
                method = "eqm", cross.val = "loo",
                precipitation = precip,
                wet.threshold = 1,
                join.members = TRUE)
  }) 
names(data.bc.cross) <- names(data)
                            
# Bias correction without cross-validation
data.bc <- lapply(1:length(data), function(v)  {
    pre <- FALSE
    if (names(data)[v] == "pr") pre <- TRUE
    biasCorrection(y = obs.data[[v]], x = data[[v]], 
                 method = "eqm",
                 precipitation = pre,
                 wet.threshold = 1,
                 join.members = TRUE)
}) 
names(data.bc) <- names(data) 

#### 4.3.4. Save results

In [ ]:
# save Rdata (*.rda file)
save(data, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_raw.rda"))
save(data.bc.cross, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BCcross.rda"))
save(data.bc, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BC.rda"))

########## BUILD FINAL DATA AND EXPORT ACCORDING TO THE WATExR ARCHIVE DESIGN -----------------------
## SEE the proposal for the WATExR Archive Design in:                                            
## https://docs.google.com/document/d/1yzNtw9W_z_ziPQ6GrnSgD9ov5O1swnohndDTAWOgpwc/edit

# Select the object to export (can be 'data.bc', 'data.bc.cross' or 'data')
datatoexport <- data.bc

# Collect some common metadata (e.g. from variable uas)
dates <- datatoexport[[1]]$Dates
xycoords <- getCoordinates(datatoexport[[1]])

# Give format to dates
yymmdd <- as.Date(dates$start)
hhmmss <- format(as.POSIXlt(dates$start), format = "%H:%M:%S") 

# Define metadata to generate the file name
ClimateModelName <- dataset
ExperimentName <- "seasonal"
freq <- "day"

# Save a single file for each member
for (i in mem) {
  # Build data.frame for a single member
  single.member <- lapply(datatoexport, function(x) subsetGrid(x, members = i))
  single.member <- lapply(single.member, function(x) x$Data)
  # Remove unwanted variables
  single.member["rsds"] <- NULL
  single.member["rlds"] <- NULL
  # data.frame creation
  df <- data.frame(c(list("dates1" = yymmdd, "dates2" = hhmmss)), single.member)
  if (i < 10) {
    member <- paste0("member0", i, sep = "", collapse = NULL)
  } else {
    member <- paste0("member", i, sep = "", collapse = NULL)
  }    
  startTime <- format(as.POSIXlt(yymmdd[1]), format = "%Y%m%d")
  endTime <- format(tail(as.POSIXlt(yymmdd), n = 1), format = "%Y%m%d")
  dirName <- paste0(dir.data, lake_id, "/CLIMATE/", lake_id, "_", institution, "_", ClimateModelName, "_", ExperimentName, "_", member, "_", freq,"_", startTime, "-", endTime, "/", sep = "", collapse = NULL)
  dir.create(dirName, showWarnings = TRUE, recursive = TRUE, mode = "0777")
  write.table(df, paste0(dirName, "meteo_file.dat", sep = "", collapse = NULL), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
}

### 4.4. Late summer

#### 4.4.1. Select dataset

In [ ]:
options(java.parameters = "-Xmx8000m")

# Define the members
mem <- 1:15
# Define the lead month
lead.month <- 0
# Define period and season
season <- c(8,9,10)  # Late summer

# Define the dataset 
dataset <- "System4_seasonal_15" # or "CFSv2_seasonal"

# Check available variables in the dataset (System4)  
di <- dataInventory("http://www.meteo.unican.es/tds5/dodsC/system4/System4_Seasonal_15Members.ncml") # or "http://meteo.unican.es/tds5/dodsC/cfsrr/CFSv2_Seasonal.ncml"
names(di)

#### 4.4.2. Data loading and transformation

In [ ]:
# Path to the observational data (change to your local path).
dir.Rdata.obs <- "/home/jovyan/projects/watexr/Rdata/PIK_Obs-EWEMBI_1_2_3_4_5_6_7_8_9_10_11_12_uas_vas_ps_tas_pr_rsds_rlds_hurs_petH.rda"
obs.data <- get(load(dir.Rdata.obs))

# Define the variables to be loaded (the same as in the observational data, 
# except clould cover (cc) and evapotranspiration (petH))
sapply(obs.data, function(x) getVarNames(x)) # to check the variables in the observational data.
variables <- c("uas", "vas", "ps", "tas", "pr", "rsds", "rlds", "hurs")

# Define daily aggregation function for each variable selected
aggr.fun <- c("mean", "mean", "mean", "mean", "sum", "mean", "mean", "mean")

# Load seasonal forecast data (System4 or CFS) with function loadECOMS
# Data is loaded in a loop (función lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable
data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
                                                          members = mem, leadMonth = lead.month,
                                                          lonLim = lonLim, latLim = latLim, season = season, 
                                                          time = "DD", aggr.d = aggr.fun[x]))
names(data.prelim) <- variables

# Bilinear interpolation of the data to the location of the lake
data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
                                                          method = "bilinear", 
                                                          bilin.method = "akima"))

# Convert pressure units to millibars with function udConvertGrid from package convertR.
data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

## Compute cloud cover with function rad2cc
#clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
#clt$Variable$varName <- "cc"
#
## Put all variables together
#data <- c(data.interp, "cc" = list(clt))
data <- data.interp

############################################################################################
############### RUN THE FOLLOWING CODE CHUNK IF YOU NEED POTENTIAL EVAPOTRANSPIRATION ######
# Load needed variables 
tasmin <- loadECOMS(dataset, var = "tasmin", years = years, 
                    lonLim = lonLim, latLim = latLim, 
                    leadMonth = lead.month, members = mem,
                    season = season, time = "DD", aggr.d = "min")
tasmax <- loadECOMS(dataset, var = "tasmax", years = years, 
                    lonLim = lonLim, latLim = latLim, 
                    leadMonth = lead.month, members = mem,
                    season = season, time = "DD", aggr.d = "max")

# Compute potential evapotranspiration with function petGrid from package drought4R
# For daily data the implemented method is hargreaves-samani (See ?petGrid for details):
petH <- petGrid(tasmin = tasmin, 
                tasmax = tasmax,
                method = "hargreaves-samani")

# bilinear interpolation 
petH.interp <- interpGrid(petH, new.coordinates = lake, method = "bilinear", bilin.method = "akima")
petH.interp$Variable$varName <- "petH"

# Put all variables together
data <- c(data, "petH" = list(petH.interp))
###################### END OF THE CHUNK ####################################################
############################################################################################

#### 4.4.3. Bias correction

In [ ]:
# Subset all datasets to the same Dates as the hindcast precipitation. Note that we compute daily accumulated 
# precipitation, for this reason this variable has no value for the first day of every season.  
if (sum(names(data)=="pr")>0){
  data <- lapply(1:length(data), function(x)  {intersectGrid(data[[x]], data[[which(names(data)=="pr")]], type = "temporal", which.return = 1)}) 
  names(data) <- sapply(data, function(x) getVarNames(x))
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))
} else{
  obs.data <- lapply(1:length(obs.data), function(x)  {intersectGrid(obs.data[[x]], data[[x]], type = "temporal", which.return = 1)}) 
  names(obs.data) <- sapply(obs.data, function(x) getVarNames(x))  
}

# Check variable consistency
if (!identical(names(obs.data), names(data))) stop("variables in obs.data and data (seasonal forecast) do not match.")

# Bias correction with leave-one-year-out ("loo") cross-validation
# type ?biasCorrection in R for more info about the parameter settings for bias correction.
data.bc.cross <- lapply(1:length(data), function(x)  {
    precip <- FALSE
    if (names(data)[x] == "pr") precip <- TRUE
    biasCorrection(y = obs.data[[x]], x = data[[x]], 
                method = "eqm", cross.val = "loo",
                precipitation = precip,
                wet.threshold = 1,
                join.members = TRUE)
  }) 
names(data.bc.cross) <- names(data)
                            
# Bias correction without cross-validation
data.bc <- lapply(1:length(data), function(v)  {
    pre <- FALSE
    if (names(data)[v] == "pr") pre <- TRUE
    biasCorrection(y = obs.data[[v]], x = data[[v]], 
                 method = "eqm",
                 precipitation = pre,
                 wet.threshold = 1,
                 join.members = TRUE)
}) 
names(data.bc) <- names(data) 

#### 4.4.4. Save results

In [ ]:
# save Rdata (*.rda file)
save(data, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_raw.rda"))
save(data.bc.cross, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BCcross.rda"))
save(data.bc, file = paste0(dir.Rdata, dataset, "_", paste0(season, collapse = "_"), "_", paste0(names(data), collapse = "_"), "_BC.rda"))

########## BUILD FINAL DATA AND EXPORT ACCORDING TO THE WATExR ARCHIVE DESIGN -----------------------
## SEE the proposal for the WATExR Archive Design in:                                            
## https://docs.google.com/document/d/1yzNtw9W_z_ziPQ6GrnSgD9ov5O1swnohndDTAWOgpwc/edit

# Select the object to export (can be 'data.bc', 'data.bc.cross' or 'data')
datatoexport <- data.bc

# Collect some common metadata (e.g. from variable uas)
dates <- datatoexport[[1]]$Dates
xycoords <- getCoordinates(datatoexport[[1]])

# Give format to dates
yymmdd <- as.Date(dates$start)
hhmmss <- format(as.POSIXlt(dates$start), format = "%H:%M:%S") 

# Define metadata to generate the file name
ClimateModelName <- dataset
ExperimentName <- "seasonal"
freq <- "day"

# Save a single file for each member
for (i in mem) {
  # Build data.frame for a single member
  single.member <- lapply(datatoexport, function(x) subsetGrid(x, members = i))
  single.member <- lapply(single.member, function(x) x$Data)
  # Remove unwanted variables
  single.member["rsds"] <- NULL
  single.member["rlds"] <- NULL
  # data.frame creation
  df <- data.frame(c(list("dates1" = yymmdd, "dates2" = hhmmss)), single.member)
  if (i < 10) {
    member <- paste0("member0", i, sep = "", collapse = NULL)
  } else {
    member <- paste0("member", i, sep = "", collapse = NULL)
  }    
  startTime <- format(as.POSIXlt(yymmdd[1]), format = "%Y%m%d")
  endTime <- format(tail(as.POSIXlt(yymmdd), n = 1), format = "%Y%m%d")
  dirName <- paste0(dir.data, lake_id, "/CLIMATE/", lake_id, "_", institution, "_", ClimateModelName, "_", ExperimentName, "_", member, "_", freq,"_", startTime, "-", endTime, "/", sep = "", collapse = NULL)
  dir.create(dirName, showWarnings = TRUE, recursive = TRUE, mode = "0777")
  write.table(df, paste0(dirName, "meteo_file.dat", sep = "", collapse = NULL), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
}